In [13]:
import numpy as np 
import pandas as pd 


In [14]:
import json

In [3]:
pip install response

Note: you may need to restart the kernel to use updated packages.


In [1]:
import response

In [3]:
import requests

## Converting the API request for AE into Dataframe

In [4]:
ae_data= requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/105792/list")

In [5]:
print(ae_data.status_code)

200


In [8]:
print(ae_data.json())


{'status': 'success', 'data': [{'studyid': 'StudyHack', 'siteid': 5538, 'subjid': 55385001, 'subjectid': 105792, 'visitindex': 1, 'visitnam': 'AECM', 'formname': 'AE202', 'formid': 38768, 'formidx': 43, 'formindex': 552000000000000.0, 'dov': '', 'aemeyn': '', 'aemeyn_c': '', 'aecat': 'ADVERSE EVENT', 'aecat_c': 1, 'aeterm': 'AST INCREASED', 'aemedrg': '', 'aeinpkg': None, 'aestdat': '25-Jun-18', 'aestdat_dts': '6/25/2018', 'aestdat_nd': '', 'aestdme': '', 'aestdme_c': None, 'aeendat': '2-Jul-18', 'aeendat_dts': '7/2/2018', 'aeendat_nd': '', 'aedis': 'NO', 'aedis_c': 'N', 'aemerae': '', 'aemerae_c': '', 'aespid': 43, 'aendgiv': 'NO', 'aendgiv_c': 'N', 'aecmgiv': 'NO', 'aecmgiv_c': 'N', 'aecmgiv_nd': '', 'aerefid': None, 'compterm': 'AST INCREASED', 'aellt': 'AST increased', 'aedecod': 'Aspartate aminotransferase increased', 'aehlt': 'Liver function analyses', 'aehlgt': 'Hepatobiliary investigations', 'aesoc': 'Investigations', 'aemedrg_c': None, 'aespid_1': None, 'aeyn': 'YES', 'aeyn_c'

In [34]:
ae_df = pd.json_normalize(ae_data.json(), record_path=['data'],meta =['status','request_id'])

In [68]:
ae_df.head()

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,...,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout
0,StudyHack,5538,55385001,105792,1,AECM,AE202,38768,43,5.520000e+14,...,NaN,,2.0,DOSE NOT CHANGED,1.0,CONCOMITANT DRUG TREATMENT,,,3,RECOVERED/RESOLVED
1,StudyHack,5538,55385001,105792,1,AECM,AE202,38768,44,5.520000e+14,...,NaN,,2.0,DOSE NOT CHANGED,1.0,CONCOMITANT DRUG TREATMENT,,,3,RECOVERED/RESOLVED
2,StudyHack,5538,55385001,105792,1,AECM,AE202,38768,45,5.520000e+14,...,NaN,,2.0,DOSE NOT CHANGED,1.0,CONCOMITANT DRUG TREATMENT,,,3,RECOVERED/RESOLVED
3,StudyHack,5538,55385001,105792,1,AECM,AE202,38768,46,5.520000e+14,...,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
4,StudyHack,5538,55385001,105792,1,AECM,AE202,38768,47,5.520000e+14,...,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED


# Converting the API request for CM subject list into Dataframe

In [79]:
cm_subj_data= requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/105792/list")

In [61]:
print(cm_subj_data.status_code)

200


In [62]:
print(cm_subj_data.json())

{'status': 'success', 'data': [{'studyid': 'StudyHack', 'siteid': 5538, 'subjid': 55385001, 'subjectid': 105792, 'visitindex': 1, 'visitnam': 'AECM', 'formname': 'AE202', 'formid': 38768, 'formidx': 43, 'formindex': 552000000000000.0, 'dov': '', 'aemeyn': '', 'aemeyn_c': '', 'aecat': 'ADVERSE EVENT', 'aecat_c': 1, 'aeterm': 'AST INCREASED', 'aemedrg': '', 'aeinpkg': None, 'aestdat': '25-Jun-18', 'aestdat_dts': '6/25/2018', 'aestdat_nd': '', 'aestdme': '', 'aestdme_c': None, 'aeendat': '2-Jul-18', 'aeendat_dts': '7/2/2018', 'aeendat_nd': '', 'aedis': 'NO', 'aedis_c': 'N', 'aemerae': '', 'aemerae_c': '', 'aespid': 43, 'aendgiv': 'NO', 'aendgiv_c': 'N', 'aecmgiv': 'NO', 'aecmgiv_c': 'N', 'aecmgiv_nd': '', 'aerefid': None, 'compterm': 'AST INCREASED', 'aellt': 'AST increased', 'aedecod': 'Aspartate aminotransferase increased', 'aehlt': 'Liver function analyses', 'aehlgt': 'Hepatobiliary investigations', 'aesoc': 'Investigations', 'aemedrg_c': None, 'aespid_1': None, 'aeyn': 'YES', 'aeyn_c'

In [69]:
cm_subj_df = pd.json_normalize(cm_subj_data.json(), record_path=['data'],meta =['status','request_id'])

In [83]:
cm_subj_df['subjectid'].value_counts()

105792    66
Name: subjectid, dtype: int64

# Converting the API request for AE subject list into Dataframe

In [89]:
ae_subj_data= requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list")

In [72]:
print(ae_subj_data.json())

{'status': 'success', 'data': [76809, 105792, 117691, 120205, 130301, 132645, 138700, 156724, 166621, 168918, 205861, 226107, 226615, 231407, 265932, 276449, 299197, 303307, 303317, 305637, 330825, 381416, 386442, 386909, 386924, 388081, 399669, 409256, 409273, 412604, 468784, 475581, 478578, 480428, 500230, 504075, 511238, 542409, 570992, 594397, 601829, 603656, 604835, 609627, 621769, 643495, 643519, 672534, 678844, 678855, 686951, 688628, 743834, 750185, 763795, 770963, 799218, 833393, 844909, 851940, 868580, 868623, 923113, 930141, 954328, 959501, 970072, 970104, 978125, 987201, 987462, 997179, 1002549, 1026894, 1026905, 1027241, 1038656, 1048183, 1049071, 1068949, 1092393, 1100385, 1107507, 1111479, 1126734, 1130157, 1130189, 1131738, 1135512, 1142200, 1146813, 1168766, 1168790, 1230976, 1253180, 1260845, 1260871, 1268747, 1350622, 1350640, 1370986, 1384439, 1387198, 1389996, 1393002, 1436157, 1453728, 1469249, 1494086, 1501322, 1505669, 1512798, 1513959, 1517961, 1525878, 1539183

In [73]:
ae_subj_df = pd.json_normalize(ae_subj_data.json(), record_path=['data'],meta =['status','request_id'])

In [74]:
ae_subj_df.head()

,0,status,request_id
0,76809,success,c994937c-4230-4d24-b97e-22f176970a51
1,105792,success,c994937c-4230-4d24-b97e-22f176970a51
2,117691,success,c994937c-4230-4d24-b97e-22f176970a51
3,120205,success,c994937c-4230-4d24-b97e-22f176970a51
4,130301,success,c994937c-4230-4d24-b97e-22f176970a51


In [76]:
ae_subj_df['status'].value_counts()

success    728
Name: status, dtype: int64

###  In the above API's, got the data from both AE api. For CM, subject list api has the required details and using same AE subject ids' for CM data

### Fetching the AE data by looping subject ids'

In [183]:
ae_ls = ae_subj_df.loc[:,0].tolist()[0:10]

In [184]:
len(ae_ls)

10

In [185]:
df = []
for i in ae_ls: 
    if i is not None: 
        temp =requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/"+str(i)+"/list")
        df.append(temp.json())
    else:
        continue
        
        

In [165]:
df

[{'status': 'success',
  'data': [{'studyid': 'StudyHack',
    'siteid': 5173,
    'subjid': 51735001,
    'subjectid': 76809,
    'visitindex': 1,
    'visitnam': 'AECM',
    'formname': 'AE202',
    'formid': 38768,
    'formidx': 4,
    'formindex': 555000000000000.0,
    'dov': '',
    'aemeyn': '',
    'aemeyn_c': '',
    'aecat': 'ADVERSE EVENT',
    'aecat_c': 1,
    'aeterm': 'BONE MARROW ASPIRATION SITE PAIN',
    'aemedrg': '',
    'aeinpkg': None,
    'aestdat': '26-Apr-18',
    'aestdat_dts': '4/26/2018',
    'aestdat_nd': '',
    'aestdme': '',
    'aestdme_c': None,
    'aeendat': '30-Apr-18',
    'aeendat_dts': '4/30/2018',
    'aeendat_nd': '',
    'aedis': 'NO',
    'aedis_c': 'N',
    'aemerae': '',
    'aemerae_c': '',
    'aespid': 4,
    'aendgiv': 'NO',
    'aendgiv_c': 'N',
    'aecmgiv': 'YES',
    'aecmgiv_c': 'Y',
    'aecmgiv_nd': '',
    'aerefid': None,
    'compterm': 'BONE MARROW ASPIRATION SITE PAIN',
    'aellt': 'Bone marrow aspiration site pain',
    

In [186]:
final_ae_df = pd.json_normalize(df, record_path=['data'],meta =['status','request_id'])

In [187]:
final_ae_df['subjectid'].value_counts()

117691    104
105792     66
120205     61
76809      40
166621     38
138700     30
130301     25
132645     25
156724     24
168918     12
Name: subjectid, dtype: int64

In [188]:
len(final_ae_df)

425

### Fetching the CM data by looping subject ids'

In [189]:
cm_df = []
for i in ae_ls: 
    if i is not None: 
        temp =requests.get("https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/"+str(i)+"/list")
        cm_df.append(temp.json())
    else:
        continue

In [190]:
final_cm_df = pd.json_normalize(cm_df, record_path=['data'],meta =['status','request_id'])

In [191]:
final_cm_df['subjectid'].value_counts()

138700    121
76809      99
166621     98
130301     67
132645     52
168918     30
Name: subjectid, dtype: int64

In [201]:
len(final_cm_df)

467

#### Converting object format into float - final_cm_df['cmaeno']

In [231]:
final_ae_df['aespid'].dtypes

dtype('float64')

In [232]:
final_cm_df['cmaeno'].dtypes

dtype('O')

In [241]:
final_cm_df['cmaeno'] = pd.to_numeric(final_cm_df['cmaeno'],errors= 'coerce').fillna(0,downcast= 'infer')

In [238]:
final_ae_df = final_ae_df.drop_duplicates()

In [239]:
final_cm_df = final_cm_df.drop_duplicates()

### Combining both AE and CM to find type errors

In [242]:
final_df = pd.merge(final_cm_df, final_ae_df
                    
        
                    
                    ,how = 'inner' ,right_on = ['subjectid','aespid'], left_on = ['subjectid','cmaeno'] )

In [197]:
Removed the Adverse events who never gets into medication using the above join functions

In [245]:
pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26817 sha256=ee5b6c655f25e888af9438a7819df9d1d05b46ea45dcbb7af1f4239860c52a22
  Stored in directory: /private/var/folders/70/34fdtr317nv47xk98x74p1dw0000gp/T/pip-ephem-wheel-cache-aw4ryl4q/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [246]:
from pandasql import sqldf 

In [253]:
final_df.to_csv("final_df.csv")

In [247]:
# Checking Medications are given and Adverse Event occur don't match

In [248]:
check1 = sqldf(select   from final_df where subjectid)

,studyid_x,siteid_x,subjid_x,subjectid,visitid,visitidx,visitnam_x,formname_x,formid_x,formidx_x,...,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout,status_y,request_id_y
0,StudyHack,5173,51735001,76809,38942,1,AECM,CM201,38623,76,...,2.0,DOSE NOT CHANGED,3.0,OTHER,,INTRATHECAL CHEMOTHERAPY,3.0,RECOVERED/RESOLVED,success,c098277e-4d64-461f-a519-6fc949847930
1,StudyHack,5173,51735001,76809,38942,1,AECM,CM201,38623,78,...,2.0,DOSE NOT CHANGED,3.0,OTHER,,INTRATHECAL CHEMOTHERAPY,3.0,RECOVERED/RESOLVED,success,c098277e-4d64-461f-a519-6fc949847930
2,StudyHack,5173,51735001,76809,38942,1,AECM,CM201,38623,85,...,2.0,DOSE NOT CHANGED,3.0,OTHER,,INTRATHECAL CHEMOTHERAPY,3.0,RECOVERED/RESOLVED,success,c098277e-4d64-461f-a519-6fc949847930
3,StudyHack,5173,51735001,76809,38942,1,AECM,CM201,38623,80,...,2.0,DOSE NOT CHANGED,3.0,OTHER,,Decreased intake.,3.0,RECOVERED/RESOLVED,success,c098277e-4d64-461f-a519-6fc949847930
4,StudyHack,5173,51735001,76809,38942,1,AECM,CM201,38623,504,...,2.0,DOSE NOT CHANGED,3.0,OTHER,,Decreased intake.,3.0,RECOVERED/RESOLVED,success,c098277e-4d64-461f-a519-6fc949847930
